In [1]:
!export MXNET_CPU_WORKER_NTHREADS=24
!export MXNET_ENGINE_TYPE=ThreadedEnginePerDevice

In [ ]:
!CUDA_VISIBLE_DEVICES='0' python3 -u train.py --network r100 --loss arcface --dataset casia --ckpt 2

gpu num: 1
prefix ./models/r100-arcface-casia/model
image_size [112, 112]
num_classes 10575
Called with argument: Namespace(batch_size=64, ckpt=2, ctx_num=1, dataset='casia', frequent=20, image_channel=3, kvstore='device', loss='arcface', lr=0.001, lr_steps='100000,160000,220000', models_root='./models', mom=0.9, network='r100', per_batch_size=64, pretrained='', pretrained_epoch=1, rescale_threshold=0, verbose=1000, wd=0.0005) {'bn_mom': 0.9, 'workspace': 256, 'emb_size': 512, 'ckpt_embedding': False, 'net_se': 0, 'net_act': 'prelu', 'net_unit': 3, 'net_input': 1, 'net_blocks': [1, 4, 6, 2], 'net_output': 'E', 'net_multiplier': 1.0, 'val_targets': ['lfw-custom', 'lfw'], 'ce_loss': True, 'fc7_lr_mult': 1.0, 'fc7_wd_mult': 1.0, 'fc7_no_bias': False, 'max_steps': 0, 'data_rand_mirror': True, 'data_cutoff': False, 'data_color': 0, 'data_images_filter': 0, 'count_flops': True, 'memonger': False, 'loss_name': 'margin_softmax', 'loss_s': 64.0, 'loss_m1': 1.0, 'loss_m2': 0.5, 'loss_m3': 0.0, '

In [2]:
!CUDA_VISIBLE_DEVICES='0' python3 -u train.py --network r100 --loss arcface --dataset casia_masked --ckpt 2 --pretrained models/r100-arcface-casia_masked/model --pretrained-epoch 1 

gpu num: 1
prefix ./models/r100-arcface-casia_masked/model
image_size [112, 112]
num_classes 10575
Called with argument: Namespace(batch_size=32, ckpt=2, ctx_num=1, dataset='casia_masked', frequent=20, image_channel=3, kvstore='device', loss='arcface', lr=0.001, lr_steps='100000,160000,220000', models_root='./models', mom=0.9, network='r100', per_batch_size=32, pretrained='models/r100-arcface-casia_masked/model', pretrained_epoch=14, rescale_threshold=0, verbose=500, wd=0.0005) {'bn_mom': 0.9, 'workspace': 256, 'emb_size': 512, 'ckpt_embedding': True, 'net_se': 0, 'net_act': 'prelu', 'net_unit': 3, 'net_input': 1, 'net_blocks': [1, 4, 6, 2], 'net_output': 'E', 'net_multiplier': 1.0, 'val_targets': ['lfw'], 'ce_loss': True, 'fc7_lr_mult': 1.0, 'fc7_wd_mult': 1.0, 'fc7_no_bias': False, 'max_steps': 0, 'data_rand_mirror': True, 'data_cutoff': False, 'data_color': 0, 'data_images_filter': 0, 'count_flops': True, 'memonger': False, 'loss_name': 'margin_softmax', 'loss_s': 64.0, 'loss_m1': 1

KeyboardInterrupt: 

In [28]:
!CUDA_VISIBLE_DEVICES='0' python3 ../src/eval/verification.py --data-dir /storage/MysterioNet/datasets/smdf/original/casia --model models/r100-arcface-casia_masked/model,14 --target lfw-masked

image_size [112, 112]
model number 1
loading models/r100-arcface-casia_masked/model 14
^C


KeyboardInterrupt: 

In [16]:
!ls /storage/MysterioNet/datasets/smdf/original/casia/*.bin

/storage/MysterioNet/datasets/smdf/original/casia/lfw-custom.bin
/storage/MysterioNet/datasets/smdf/original/casia/lfw-masked.bin
/storage/MysterioNet/datasets/smdf/original/casia/lfw.bin


In [3]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [3]:
!pip3 install opencv-python

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [4]:
!pip3 install scikit-learn

You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [5]:
!pip3 install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6349 sha256=93f43e7441f9d17e484b45ed415c2ef72d3956cf2ff94e70e687f002ecbeee83
  Stored in directory: /root/.cache/pip/wheels/5d/79/e4/4e55effe206295359b37e0f9db3e68a1197ba396682807dadb
Successfully built easydict
You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [6]:
!pip3 install contextvars

     |████████████████████████████████| 98 kB 13.3 MB/s eta 0:00:01
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7664 sha256=eed5292114111c56f825357ecacae41fb4f48e0f1b3282cef7da16b9301983c1
  Stored in directory: /root/.cache/pip/wheels/41/11/53/911724983aa48deb94792432e14e518447212dd6c5477d49d3
Successfully built contextvars
You should consider upgrading via the '/usr/bin/python3.6 -m pip install --upgrade pip' command.


In [1]:
batch_size = 32
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), 'eval'))
import verification
import mxnet as mx
import numpy as np
import pickle
from mxnet import ndarray as nd
ctx = mx.gpu(0)
image_size=[112,112]
batch_size = 32
sym, arg_params, aux_params = mx.model.load_checkpoint("models/r100-arcface-casia_masked/model", 1)
all_layers = sym.get_internals()
sym = all_layers['fc1_output']
model = mx.mod.Module(symbol=sym, context=ctx, label_names = None)

model.bind(data_shapes=[('data', (batch_size, 3,112,112))])
model.set_params(arg_params, aux_params)

data_set = verification.load_bin("/storage/MysterioNet/datasets/smdf/original/casia/lfw-masked.bin", image_size)

loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
loading bin 13000
loading bin 14000
loading bin 15000
loading bin 16000
loading bin 17000
loading bin 18000
loading bin 19000
loading bin 20000
(20126, 3, 112, 112)


In [8]:
acc1, std1, acc2, std2, xnorm, embeddings_list = verification.test(data_set, model, batch_size, 10, None, None)

testing verification..
(20126, 512)
infer time 131.38679000000008
Threshold:  0.0074205
Threshold:  0.0074205
Threshold:  0.0074205
Threshold:  0.0074205
Threshold:  0.007892
Threshold:  0.007338
Threshold:  0.007225333333333334
Threshold:  0.007169
Threshold:  0.007419


In [9]:
print('XNorm: %f' % ( xnorm))
print('Accuracy: %1.5f+-%1.5f' % ( acc1, std1))
print('Accuracy-Flip: %1.5f+-%1.5f' % ( acc2, std2))

XNorm: 4.292508
Accuracy: 0.00000+-0.00000
Accuracy-Flip: 0.57584+-0.19385


In [132]:
from matplotlib import pyplot as plt
import numpy as np
def render_as_image(a):
    img = a.asnumpy() # convert to numpy array
    img = img.transpose((1, 2, 0))
    img = img.astype(np.uint8)  # use uint8 (0-255)

    plt.imshow(img)
    plt.show()

In [5]:
data_list = data_set[0]
issame_list = data_set[1]

_label = nd.ones( (batch_size,) )
# no flip type
data = data_list[0]
embeddings = None
ba = 0
embeddings_list = []

In [6]:
import numpy as np
for i in range( len(data_list) ):
    data = data_list[i]
    embeddings = None
    ba = 0
    while ba<data.shape[0]:
        bb = min(ba+batch_size, data.shape[0])
        count = bb-ba
        _data = nd.slice_axis(data, axis=0, begin=bb-batch_size, end=bb)
        #print(_data.shape, _label.shape)
        db = mx.io.DataBatch(data=(_data,), label=(_label,))
        model.forward(db, is_train=False)
        net_out = model.get_outputs()
        #_arg, _aux = model.get_params()
        #__arg = {}
        #for k,v in _arg.iteritems():
        #  __arg[k] = v.as_in_context(_ctx)
        #_arg = __arg
        #_arg["data"] = _data.as_in_context(_ctx)
        #_arg["softmax_label"] = _label.as_in_context(_ctx)
        #for k,v in _arg.iteritems():
        #  print(k,v.context)
        #exe = sym.bind(_ctx, _arg ,args_grad=None, grad_req="null", aux_states=_aux)
        #exe.forward(is_train=False)
        #net_out = exe.outputs
        _embeddings = net_out[0].asnumpy()
        #print(_embeddings.shape)
        if embeddings is None:
            embeddings = np.zeros( (data.shape[0], _embeddings.shape[1]) )
        embeddings[ba:bb,:] = _embeddings[(batch_size-count):,:]
        ba = bb
    embeddings_list.append(embeddings)

In [7]:
_xnorm = 0.0
_xnorm_cnt = 0
for embed in embeddings_list:
    for i in range(embed.shape[0]):
        _em = embed[i]
        _norm=np.linalg.norm(_em)
        #print(_em.shape, _norm)
        _xnorm+=_norm
        _xnorm_cnt+=1
    _xnorm /= _xnorm_cnt

In [8]:
embeddings = embeddings_list[0] + embeddings_list[1]

In [9]:
import sklearn
embeddings = sklearn.preprocessing.normalize(embeddings)
print(embeddings.shape)

(38354, 512)


In [10]:
thresholds = np.arange(0, 4, 0.01)

In [11]:
embeddings1 = embeddings[0::2]
embeddings2 = embeddings[1::2]

In [12]:
actual_issame = issame_list

In [13]:
nrof_pairs = min(len(actual_issame), embeddings1.shape[0])

In [14]:
from sklearn.model_selection import KFold
class LFold:
  def __init__(self, n_splits = 2, shuffle = False):
    self.n_splits = n_splits
    if self.n_splits>1:
      self.k_fold = KFold(n_splits = n_splits, shuffle = shuffle)

  def split(self, indices):
    if self.n_splits>1:
      return self.k_fold.split(indices)
    else:
      return [(indices, indices)]

nrof_folds=10
nrof_thresholds = len(thresholds)
k_fold = LFold(n_splits=nrof_folds, shuffle=False)

In [15]:
val = np.zeros(nrof_folds)
far = np.zeros(nrof_folds)

In [16]:
diff = np.subtract(embeddings1, embeddings2)
dist = np.sum(np.square(diff),1)

In [17]:
indices = np.arange(nrof_pairs)

In [18]:
x = list(k_fold.split(indices))

In [19]:
(train_set, test_set) = x[0]

In [20]:
far_train = np.zeros(nrof_thresholds)

In [29]:
train_dist = dist[test_set]

In [30]:
train_issame = np.array(actual_issame)[test_set]

In [31]:
train_dist

array([0.10807752, 0.53395856, 0.53395856, ..., 0.37931892, 0.62126807,
       1.51095461])

In [32]:
np.array(actual_issame)

array([ True,  True,  True, ..., False, False, False])

In [33]:
train_set.shape

(17259,)

In [34]:
test_set

array([   0,    1,    2, ..., 1915, 1916, 1917])

In [35]:
predict_issame = np.less(train_dist, 0.15701524999999997)
tp = np.sum(np.logical_and(predict_issame, train_issame))
fp = np.sum(np.logical_and(predict_issame, np.logical_not(train_issame)))
tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(train_issame)))
fn = np.sum(np.logical_and(np.logical_not(predict_issame), train_issame))
print(tp, " ", fp," ", tn," ", fn)

71   0   0   1847


In [36]:
np.sum(train_issame)
n_diff = np.sum(np.logical_not(train_issame))

In [37]:
np.sum(train_issame)

1918

In [38]:
np.sum(predict_issame)

71